In [1]:
import requests
from bs4 import BeautifulSoup
import time 
import random 
import re
from fake_useragent import UserAgent
import datetime
from elasticsearch import Elasticsearch
from tqdm import tqdm

In [2]:
class KanobuArticle:
    def __init__(self):
        self.date=""
        self.time=""
        self.words=""
        self.rubr=""
        self.hashtag=""
        self.author=""
        self.head=""
        self.text=""
        
    # Конвертация в JSON.
    def toJSON(self):
        res='{"date":"'+self.date+'", "time":"'+self.time+'", "words":"'+self.words+'", "rubrics":"'+self.rubr+'", "hashtag":"'+self.hashtag+'", "title":"'
        +self.head+'", "author":"'+self.author+'","text":"'
        res+=self.text.replace('"', '\\"')+'"}'
        return res

    # Конвертация в словарь.
    def toDict(self):
        res={"date":self.date, "time":self.time, "rubrics":self.rubr, "hashtag":self.hashtag, \
             "title":self.head, "words":self.words, "author":self.author,"text":self.text.replace('"', '\\"')}
        return res

In [3]:
def GetPagesArticles(number_of_pages):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'}
    url = 'https://kanobu.ru/news/?page='
    links = []
    for page in range(1, number_of_pages+1):
        r = requests.get(url + str(page), headers = headers)
        time.sleep(random.randint(1,6))
        content = r.content
        html = content.decode('UTF-8')
        soup = BeautifulSoup(html)
        
        for link in soup.find_all('a', attrs={'class':'c-item_foot d-b t-l'}):
            links.append(link.get('href'))
    
    return links    

In [4]:
def clean_text (text):
    text = text.replace('\xa0', ' ')
    text = text.replace ('\n', ' ')
    text = text.replace ('  ', ' ')
    return text

In [5]:
def GetArticles(links):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'}
    l = 'https://kanobu.ru'
    articles = []
    for link in links:
        art = KanobuArticle()
        page = requests.get(l+link, headers = headers)
        time.sleep(random.randint(1,3))
        content2 = page.content
        html2 = content2.decode('UTF-8')
        soup2 = BeautifulSoup(html2)
        
        date = soup2.find('div', attrs={'class':'c-title_footer'}).get_text()
        art.date = date.split(', ')[0]
        art.time = date.split(', ')[1]
        
        author = soup2.find('a', attrs={'class':'c-title_author'}).get_text()
        art.author = author
        
        text = soup2.find('div', attrs={'class':'c-detail_content'}).get_text()
        art.text = clean_text(text)
        
        words = len(clean_text(text).split())
        art.words = words
        
        title = soup2.find('h1', attrs={'class':'c-title_body'}).get_text()
        art.head = clean_text(title)
        
        rubric = soup2.find('div', attrs={'class':'c-title_head t-t-u'}).get_text()
        r = rubric.split(' | ')
        art.rubr = r[0]
        try:
            art.hashtag = r[1]
        except:
            pass
        #print(art)
        #print(text)
        #print(date)
        #print(author)
        print(title)
        print(words)
        #print(art.date)
        #print(r[0])
        #print(r[1])
        #print(rubric)
        articles += [art]
    return articles

In [7]:
articles = GetArticles(GetPagesArticles(5))

«Однажды в… Голливуде» может стать книгой. Об этом рассказал Тарантино
153
Представлен ультрабюджетный фитнес-трекер Redmi Band
122
Смотрим финальный трейлер Final Fantasy VII Remake
103
AnTuTu назвал самые мощные Android-смартфоны марта
137
Лидер Little Big снял клип о королеве карантина. Это заявка на «Карантиновидение-2020»
172
Когда выйдут «Тихое место 2» и «Топ Ган: Мэверик»? Paramount назвала новые даты релиза своих фильмов
158
Xiaomi Mi Air 2S: беспроводные наушники с шумоподавлением и сутками автономной работы за 4300 рублей
128
Список фигурок по «Вечным» выдал новых персонажей фильма Marvel
169
В Apex Legends стартует новое сюжетное событие. Что еще свежего
98
Обновление Windows 10 уменьшит нагрузку на процессор и место системы на диске
102
На BAFTA Games Awards 2020 назвали лучшую игру года. И это не Death Stranding
161
iPhone 9 не выйдет. Новый флагман Apple все-таки назовут iPhone SE
183
Инсайдер поделился подробностями о перезагрузке «Обители зла»
198
Режиссер «Шазама» опу

### База

In [8]:
es=Elasticsearch()

In [8]:
es

<Elasticsearch([{}])>

In [9]:
es.indices.delete(index="kanobu") 

{'acknowledged': True}

In [10]:
es.indices.create(index="kanobu")

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'kanobu'}

In [11]:
mapit={"article":{"properties":{"author":{"type":"text"},
                                "date":{"type":"text"},
                                "time":{"type":"date", "format":" HH:mm"},
                                "words":{"type":"double"},
                                "hashtag":{"type":"text","analyzer" : "russian"},
                                "rubrics":{"type":"text","analyzer" : "russian"},
                                "text":{"type":"text","analyzer" : "russian"},
                                "title":{"type":"text","analyzer" : "russian"}}}}

In [12]:
es.indices.put_mapping(index="kanobu", doc_type='article', body=mapit, include_type_name=True)

{'acknowledged': True}

In [13]:
es.indices.get_alias("*")

{'kanobu': {'aliases': {}}}

In [14]:
%%time
i = 0
for article in tqdm(articles):
    es.index(index="kanobu", doc_type='article', id = i, body=article.toDict())
    i+=1

100%|█████████████████████████████████████████████████████████████████████████████████| 90/90 [00:00<00:00, 107.12it/s]

Wall time: 860 ms


In [15]:
res = es.search(index="kanobu", body={"query": {"match_all": {}}}, size=100)
res["hits"]["hits"]

[{'_index': 'kanobu',
  '_type': 'article',
  '_id': '0',
  '_score': 1.0,
  '_source': {'date': '03 апреля 2020',
   'time': '11:35',
   'rubrics': 'Кино и сериалы',
   'hashtag': '#квентин тарантино',
   'title': '«Однажды в… Голливуде» может стать книгой. Об этом рассказал Тарантино',
   'words': 153,
   'author': 'Анастасия Яблоновская',
   'text': 'Американский кинематографист Квентин Тарантино планирует написать роман по сюжету фильма «Однажды в… Голливуде». Об этом он рассказал в недавнем выпуске подкаста Pure Cinema. Тарантино сообщил, что размышляет о том, чтобы превратить сценарии своих фильмов в книги. В основу одного из романов может лечь сюжет картины «Однажды в… Голливуде». «Я не задумывался об этом до недавнего времени. Но сейчас я много размышляю над этим. Я мог бы написать роман по „Однажды в… Голливуде“», — заявил режиссер. 10 лучших фильмов 2019 года — по версии читателей «Канобу». От «Джокера» до «Боевого ангела» Напомним, фильм «Однажды в… Голливуде» вышел в россий

### Добавление в базу новой статьи по ссылке

In [23]:
def add_article(link):
    articles = []
    art = KanobuArticle()
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'}
    page = requests.get(link, headers = headers)
    content = page.content
    html = content.decode('UTF-8')
    soup = BeautifulSoup(html)
    
    date = soup.find('div', attrs={'class':'c-title_footer'}).get_text()
    art.date = date.split(', ')[0]
    art.time = date.split(', ')[1]
        
    author = soup.find('a', attrs={'class':'c-title_author'}).get_text()
    art.author = author
        
    text = soup.find('div', attrs={'class':'c-detail_content'}).get_text()
    art.text = clean_text(text)
        
    words = len(clean_text(text).split())
    art.words = words
        
    title = soup.find('h1', attrs={'class':'c-title_body'}).get_text()
    art.head = clean_text(title)
        
    rubric = soup.find('div', attrs={'class':'c-title_head t-t-u'}).get_text()
    r = rubric.split(' | ')
    art.rubr = r[0]
    art.hashtag = r[1]
    
    #print(art)
    #print(text)
    #print(date)
    #print(author)
    #print(title)
    #print(words)
    #print(r[0])
    #print(r[1])
    
    articles += [art]
    
    res = es.search(index="kanobu", body={"query": {"match_all": {}}})
    maxid = res['hits']['total']['value']
    print('new_article index =', maxid)
    
    for article in articles:
        es.index(index="kanobu", doc_type='article', id = maxid, body=article.toDict())
        
    print('Done')

In [24]:
link = 'https://kanobu.ru/news/entoni-hopkins-syigral-nafortepiano-dlya-svoego-kota-i-pokoril-sotsseti--422323/'
add_article(link)

new_article index = 89
Done


### Мэтч по id новой статьи

In [25]:
es.get(index="kanobu", doc_type='article', id=89)

{'_index': 'kanobu',
 '_type': 'article',
 '_id': '89',
 '_version': 3,
 '_seq_no': 92,
 '_primary_term': 1,
 'found': True,
 '_source': {'date': '20 марта 2020',
  'time': '12:59',
  'rubrics': 'Кино и сериалы',
  'hashtag': '#энтони хопкинс',
  'title': 'Энтони Хопкинс сыграл на фортепиано для своего кота и покорил соцсети ',
  'words': 164,
  'author': 'Анастасия Яблоновская',
  'text': 'Умиротворяющим видео поделился с подписчиками Энтони Хопкинс. Он сыграл на фортепиано для своего кота. Судя по ролику, питомец музыкальным талантом актера доволен. «Niblo заботится о том, чтобы я оставался здоровым, а взамен требует себя развлекать», — отметил Хопкинс в подписи к видео. Баста дал концерт в онлайне. Все, чтобы спасти нас от скуки Ролик получил большую популярность в сети. На Facebook у него уже более 6 млн. просмотров. В комментариях поклонники актера назвали видео замечательным и очень милым. А еще пожелали кумиру беречь здоровье. Вчера вдохновляющим роликом поделилась с фанатами ак

## Удаление статьи по id

In [26]:
es.delete(index="kanobu",doc_type="article",id=89)

{'_index': 'kanobu',
 '_type': 'article',
 '_id': '89',
 '_version': 4,
 'result': 'deleted',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 93,
 '_primary_term': 1}

### Мэтч по заголовку

In [19]:
res = es.search(index="kanobu", body={"query": {"match": {"title": "россия"}}})
res["hits"]["hits"]

[{'_index': 'kanobu',
  '_type': 'article',
  '_id': '32',
  '_score': 3.352501,
  '_source': {'date': '02 апреля 2020',
   'time': '18:24',
   'rubrics': 'Игры',
   'hashtag': '#resident evil',
   'title': 'В России выпустят книгу об истории Resident Evil',
   'words': 219,
   'author': 'Сергей Сурепин',
   'text': 'Издательство «БОМБОРА» купило права на книгу Алекса Аниэля. Работа получила название An Itchy, Tasty History of Resident Evil. Работа блогера и менеджера компаний Brave Wave Productions и Limited Run Games состоит из хроники развития серии Resident Evil — начиная с выпуска оригинала в 1996 году и до конца 2006 года. Об этом «Канобу» сообщили представители издательства «БОМБОРА». На протяжении двух лет Алекс разговаривали с ключевыми сотрудниками «золотого состава» японской корпорации Capcom. Теперь нас ждет рассказ о внутренней истории компании: как идеи создать игру в жанре survival horror появились еще в конце 80-х, как неожиданный и беспрецедентный успех RE спас Capcom 

### Мэтч по дате

In [27]:
res = es.search(index="kanobu", body={"query": {"match": {"date": "22 марта 2020"}}}, size = 100)
res["hits"]["hits"]

[{'_index': 'kanobu',
  '_type': 'article',
  '_id': '0',
  '_score': 0.005390848,
  '_source': {'date': '03 апреля 2020',
   'time': '11:35',
   'rubrics': 'Кино и сериалы',
   'hashtag': '#квентин тарантино',
   'title': '«Однажды в… Голливуде» может стать книгой. Об этом рассказал Тарантино',
   'words': 153,
   'author': 'Анастасия Яблоновская',
   'text': 'Американский кинематографист Квентин Тарантино планирует написать роман по сюжету фильма «Однажды в… Голливуде». Об этом он рассказал в недавнем выпуске подкаста Pure Cinema. Тарантино сообщил, что размышляет о том, чтобы превратить сценарии своих фильмов в книги. В основу одного из романов может лечь сюжет картины «Однажды в… Голливуде». «Я не задумывался об этом до недавнего времени. Но сейчас я много размышляю над этим. Я мог бы написать роман по „Однажды в… Голливуде“», — заявил режиссер. 10 лучших фильмов 2019 года — по версии читателей «Канобу». От «Джокера» до «Боевого ангела» Напомним, фильм «Однажды в… Голливуде» вышел 

### Виджеты

In [29]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual

"Требования к нереляционным БД
2 - нереляционная БД
2 - красивая структура БД
2 - интерфейс позволяет класть, доставать, удалять данные (проводить операции CRUD - Create, Read, Update, Delete)
2 - два действия помимо CRUD (сортировка, группировка, агрегация, ...)
2 - зависит от БД: Redis - использование ключей, хешей, ...; Neo4j - нахождение путей не только к соседним вершинам, операции с графами; MongoDB, ElasticSearch - работа с текстами или географией; остальные БД - будем договариваться."

### 1 показ всех статей

In [126]:
button2 = widgets.Button(description='Match with options')
out = widgets.Output()

def on_button_clicked2(_):
    with out:
        clear_output()
        if menu.value == 'id':
            id_search = int(text.value)
            print(es.get(index="kanobu", doc_type='article', id=id_search))
        
        else:
            res = es.search(index="kanobu", body={"query": {"match": {menu.value : text.value}}}, size=100)
            for part in res["hits"]["hits"]:
                print(part)

In [127]:
button2.on_click(on_button_clicked2)

In [128]:
text1 = widgets.Text(
    value='',
    placeholder='Type what you want to find',
    description='Text:',
    disabled=False
)

In [129]:
menu = widgets.Dropdown(
       options=['id', 'title', 'date', 'author', 'hashtag', 'rubrics'],
       value='id',
       description='How to find?')

In [130]:
text = widgets.Text(
    value='',
    placeholder='Type what you want to find',
    description='Text:',
    disabled=False
)

In [131]:
text2 = widgets.Text(
    value='',
    placeholder='Enter the link of the article to add',
    description='Text:',
    disabled=False
)

In [132]:
order = widgets.Dropdown(
       options=['ascending', 'descending'],
       value='ascending',
       description='Order')

In [133]:
sortby = widgets.Dropdown(
       options=['_id', 'words'],
       value='_id',
       description='Sort by')

In [134]:
number = widgets.IntSlider(
    value=10,
    min=1,
    max=10000,
    step=1,
    description='How many articles?',
)



In [162]:
button3 = widgets.Button(description='Add article')
out = widgets.Output()

def on_button_clicked3(_):
    with out:
        clear_output()
        link = text2.value
        add_article(link)

In [163]:
button3.on_click(on_button_clicked3)

In [137]:
text3 = widgets.Text(
    value='',
    placeholder='Enter article id to delete',
    description='Text:',
    disabled=False
)

In [138]:
button4 = widgets.Button(description='Delete article')
out = widgets.Output()

def on_button_clicked4(_):
    with out:
        clear_output()
        id_del = int(text3.value)
        es.delete(index="kanobu",doc_type="article",id=id_del)
        print('Done')

In [139]:
button4.on_click(on_button_clicked4)

In [140]:
button5 = widgets.Button(description='Sort articles')
out = widgets.Output()

In [141]:
def on_button_clicked5(_):
    with out:
        clear_output()
        if order.value == 'ascending':
            ord = str(sortby.value)+':asc'
        elif order.value == 'descending':
            ord = str(sortby.value)+':desc'
        res = es.search(index="kanobu", body={"query": {"match_all": {}}}, sort=ord,size=number.value)
        for part in res["hits"]["hits"]:
            print(part)

In [142]:
button5.on_click(on_button_clicked5)

In [143]:
id_upd = widgets.Text(
    value='',
    placeholder='Enter your id',
    description='Text:',
    disabled=False
)

menu_upd = widgets.Dropdown(
       options=['title', 'date', 'author', 'hashtag', 'rubrics', 'text'],
       value='title',
       description='What to update?')

text_upd = widgets.Text(
    value='',
    placeholder='Enter your update',
    description='Text:',
    disabled=False
)

button_upd = widgets.Button(description='Update')
out = widgets.Output()

def on_button_clicked_upd(_):
    with out:
        clear_output()
        your_id = int(id_upd.value)
        upd_area = str(menu_upd.value)
        text = str(text_upd.value)
        es.update(index='kanobu', doc_type='article', id=your_id, body={"doc": {upd_area: text}})
        print('Done')

In [144]:
button_upd.on_click(on_button_clicked_upd)

In [164]:
box1 = widgets.VBox([menu, text, button2])
box2 = widgets.VBox([text2, button3, text3, button4])
box3 = widgets.VBox([id_upd, menu_upd, text_upd, button_upd])
box4 = widgets.VBox([sortby,order, number, button5])
widgets.VBox([box1, box2, box3, box4, out])

- поиск по Id выполняется по другому (придумать как добавить его в ту же кнопку) - <b>сделано</b>
- кнопка для добавления статьи по ссылке - <b>сделано</b>
- не хватает операции update (например, "положи по существующему индексу другую статью" - <b>сделано</b>
- нужно организовать меню с кнопками получше - <b>сделано</b>
- сортировка <b>сделано</b>
- виджет с агрегацией
- группировка 
- "2 - два действия помимо CRUD (сортировка, группировка, агрегация, ...) "
- интерфейс отдельным файлом
- внутренность интерфейса отдельным файлом
- внутренность бд отдельным файлом

In [ ]:
https://stackoverflow.com/questions/30598152/how-to-update-a-document-using-elasticsearch-py/30598673

In [ ]:
res=es.search(index = "syntagrus", body = {"size": 0, "aggs": {"max_id": {"max": {"field": "sentID" }}}})
# Получение максимального номера предложения, уже хранимого в базе.

In [181]:
res=es.search(index = "kanobu", body = {"size": 0, "aggs": {"max_id": {"max": {"field": "words" }}}})
res["aggregations"]["max_id"]["value"]

299.0

In [180]:
res2=es.search(index = "kanobu", body = {"size": 0, "aggs": {"max_id": {"avg": {"field": "words" }}}})
res2["aggregations"]["max_id"]["value"]

161.35227272727272